In [2]:
import os

from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [20]:
from uuid import uuid4

from langchain_core.documents import Document

references = ["The cat sat on the mat.", "A dog is lying on a floor."]

documents = [Document(
    page_content=ref,
    metadata={"source": "tweet"}
    )
    for ref in references]

uuids = [str(uuid4()) for _ in range(len(documents))]

db = FAISS.from_documents(documents, OllamaEmbeddings(model="nomic-embed-text",show_progress=True))
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

# Set up the local model:
local_model = "llama3"
llm = ChatOllama(model=local_model, num_predict=400,
                 stop=["<|start_header_id|>", "<|end_header_id|>", "<|eot_id|>"])

# Set up the RAG chain:
prompt_template = """
<|start_header_id|>user<|end_header_id|>
Responds user questions taking into accoun the given context, give a precise and short answer without referring to this part of prompt.
Question: {question}
Context: {context}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Querying the LLM (oviously to test here you must ask a relevant question of your data)
question = "Where is the cat?"
print(question)
print(rag_chain.invoke(question))

OllamaEmbeddings: 100%|██████████| 2/2 [00:00<00:00, 90.22it/s]


Where is the cat?


OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 170.66it/s]


The cat is on the mat!


In [10]:
retriever | format_docs

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x74e528690880>, search_kwargs={'k': 4})
| RunnableLambda(format_docs)

In [28]:
from transformers import BertTokenizer, BertModel
from bert_score import BERTScorer
# Example texts
reference = ["The cat sat on the mat.", "A dog is lying on a floor."]
candidate = ["The cat is on the mat!", "The dog is lying on the floor."]
# BERTScore calculation
scorer = BERTScorer(model_type='bert-base-uncased')
P, R, F1 = scorer.score(candidate, reference)
print(f"BERTScore Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")

BERTScore Precision: 0.7900, Recall: 0.7871, F1: 0.7885


In [25]:
# Define the reference and candidate sentences
reference = ["The cat sat on the mat."]
candidate = ["The cat is on the mat!"]

# Compute the BERTScore
P, R, F1 = scorer.score(candidate, reference)

# Print the scores
print("Precision: {:.2f}, Recall: {:.2f}, F1: {:.2f}".format(P.item(), R.item(), F1.item()))

Precision: 0.73, Recall: 0.73, F1: 0.73


In [ ]:
kfn